In [57]:
import os
import shutil
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cv2
import shutil

In [58]:
root = 'results/mnist/'
for run in os.listdir(root):
    if run == '0images':
            continue
    for folder in tqdm(os.listdir(root + run)):
        for file in os.listdir(root + run + '/' + folder):
            if file.endswith('.json'):
                number = file.replace('.json', '').replace('config-', '')
                try:
                    shutil.move(root + 'fed-job-{}.out'.format(number), root + folder)
                except:
                    pass
            if file.endswith('npy'):
                metric = file.replace('.npy', '')
                if os.path.exists(root + run + '/' + folder + '/' + metric + '.png'):
                    continue
                y = np.load(root + run + '/' + folder + '/' + file)
                x = np.arange(1, len(y) + 1)
                mean = y.cumsum() / x
                fig = plt.figure()
                plt.plot(x, y, label=metric.replace('_', ' '))
                plt.plot(x, mean, label=metric.replace('_', ' ') + ' mean')
                plt.xlabel('num of round')
                plt.grid(True)
                plt.yticks(np.arange(0, 1, 0.1))
                plt.legend()
                plt.savefig(root + run + '/' + folder + '/' + metric + '.png', bbox_inches='tight')
                plt.close(fig)

100%|██████████| 300/300 [00:00<00:00, 3060.57it/s]


In [59]:
result_map = defaultdict(list)

krum_images = []
for run in os.listdir(root):
    if run == '0images':
            continue
    for folder in os.listdir(root + run):
        if folder == '0images':
            continue
        accuracy_m = 0
        accuracy_s = 0
        for file in os.listdir(root + run + '/' + folder):
            if file.endswith('.npy'):
                results = np.load(root + run + '/' + folder + '/' + file)
                num = max(20, len(results) // 10)
                accuracy_m = np.median(results[-num:])
                deg = '0.4'
                if 'adni' in folder:
                    deg = '0'
                if deg+'--krum--clean' in folder:
                    krum_images.append(root + run + '/' + folder + '/' + file.replace('.npy', '.png'))
                accuracy_s = results[-num:].std()
                if file.startswith('backdoor'):
                    break
        result_map[folder].append((accuracy_m, accuracy_s))


path = root+'0images/krum/'
os.makedirs(path,exist_ok=True)
for i, image in enumerate(krum_images):
    shutil.copy(image, path+'{}.png'.format(i))



In [60]:
data = []
for config in result_map:
    results = result_map[config]
    results = np.median(results, axis=0)
    a = list(filter(lambda x: x, config.split('-')))
    dataset = a[0]
    non_iid = a[1]
    aggregator = a[2]
    attack = a[3]
    fraction = 0
    if len(a) > 4:
        fraction = a[4]
    data.append((dataset, non_iid, aggregator, attack, fraction, *results))

df = pd.DataFrame(data, columns=['dataset', 'non_iid', 'aggregator', 'attack', 'fraction', 'accuracy_m', 'accuracy_s'])

df.to_csv(root + '0images/raw.csv')

In [61]:
def group_and_show(df, group_by, exclusions=None):
    if exclusions:
        df = df.drop(exclusions, axis=1)
    groups = df.groupby(group_by)
    frames = []
    for i , (_, frame) in enumerate(groups):

        if i!= len(groups)-1:
            frame = frame.append(pd.Series(dtype=object), ignore_index=True)
        frames.append(frame)

    frame = pd.concat(frames).fillna('').reset_index(drop=True)

    def fill(row):
        if row.dataset == '':
            return ['background-color: #e0e2e5'] * len(row)
        return [''] * len(row)

    display(frame.style.hide_index().set_properties(**{'font-size': '.8rem'}).apply(fill, axis=1))


### Comparison of aggregators with no attacks

In [62]:
clean_results =  df[df['attack'] == 'clean']

group_and_show(clean_results, 'non_iid', ['attack', 'fraction'])
# group_and_show(clean_results, 'aggregator', ['attack', 'fraction'])

dataset,non_iid,aggregator,accuracy_m,accuracy_s
mnist,0,fedavg,0.893600,0.001805
mnist,0,krum,0.823550,0.017709
mnist,0,median,0.888000,0.002785
mnist,0,trimmedmean,0.891400,0.002312
,,,,
mnist,0.4,fedavg,0.876800,0.011807
mnist,0.4,krum,0.707650,0.025115
mnist,0.4,median,0.852350,0.017986
mnist,0.4,trimmedmean,0.872200,0.009388
,,,,


#
#
### Comparison of attacks on the baseline (fed averaging)


In [63]:
fed_avg_results =  df[df['aggregator'] == 'fedavg']
fed_avg_results = fed_avg_results[fed_avg_results['attack']!='clean']

group_and_show(fed_avg_results, 'attack', ['aggregator'])

dataset,non_iid,attack,fraction,accuracy_m,accuracy_s
mnist,0,backdoor,0.1,1.000000,0.000436
mnist,0,backdoor,0.3,1.000000,0.001200
mnist,0,backdoor,0.5,0.996000,0.000000
mnist,0.4,backdoor,0.1,1.000000,0.000600
mnist,0.4,backdoor,0.3,1.000000,0.000000
mnist,0.4,backdoor,0.5,1.000000,0.000000
mnist,0.7,backdoor,0.1,1.000000,0.000917
mnist,0.7,backdoor,0.3,0.998000,0.000866
mnist,0.7,backdoor,0.5,0.998000,0.000995
,,,,,


#
#
### Comparison of aggregators against attacks


In [64]:

defenses_results =  df[df['attack']!='clean']

for non_iid, group in defenses_results.groupby('non_iid'):
    display(HTML('<br/><h3>{}</h3>'.format('non iid degree: {}'.format(non_iid))))
    group_and_show(group.sort_values('fraction'), 'attack', ['non_iid'])



dataset,aggregator,attack,fraction,accuracy_m,accuracy_s
mnist,fedavg,backdoor,0.1,1.000000,0.000436
mnist,krum,backdoor,0.1,0.000000,0.000800
mnist,median,backdoor,0.1,0.500000,0.079454
mnist,trimmedmean,backdoor,0.1,0.994000,0.000995
mnist,median,backdoor,0.3,0.994000,0.001584
mnist,trimmedmean,backdoor,0.3,1.000000,0.000000
mnist,fedavg,backdoor,0.3,1.000000,0.001200
mnist,krum,backdoor,0.3,0.002000,0.000714
mnist,trimmedmean,backdoor,0.5,0.994000,0.000980
mnist,fedavg,backdoor,0.5,0.996000,0.000000


dataset,aggregator,attack,fraction,accuracy_m,accuracy_s
mnist,fedavg,backdoor,0.1,1.000000,0.000600
mnist,krum,backdoor,0.1,0.004000,0.011529
mnist,median,backdoor,0.1,0.463000,0.055321
mnist,trimmedmean,backdoor,0.1,0.996000,0.000995
mnist,median,backdoor,0.3,1.000000,0.000800
mnist,trimmedmean,backdoor,0.3,0.996000,0.001338
mnist,fedavg,backdoor,0.3,1.000000,0.000000
mnist,krum,backdoor,0.3,0.002000,0.004862
mnist,trimmedmean,backdoor,0.5,1.000000,0.000000
mnist,fedavg,backdoor,0.5,1.000000,0.000000


dataset,aggregator,attack,fraction,accuracy_m,accuracy_s
mnist,fedavg,backdoor,0.1,1.000000,0.000917
mnist,krum,backdoor,0.1,0.036000,0.027320
mnist,median,backdoor,0.1,0.429000,0.052380
mnist,trimmedmean,backdoor,0.1,0.996000,0.001308
mnist,median,backdoor,0.3,1.000000,0.000000
mnist,trimmedmean,backdoor,0.3,0.998000,0.000714
mnist,fedavg,backdoor,0.3,0.998000,0.000866
mnist,krum,backdoor,0.3,0.005000,0.028003
mnist,trimmedmean,backdoor,0.5,1.000000,0.000000
mnist,fedavg,backdoor,0.5,0.998000,0.000995


In [65]:

def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 1.03 * height,
                '%d' % int(np.round(height)),
                ha='center', va='bottom')


In [66]:

fed_avg_results =  df[df['aggregator'] == 'fedavg']
# fed_avg_results = fed_avg_results[fed_avg_results['attack']!='clean']
fed_avg_clean = fed_avg_results[fed_avg_results['attack']=='clean']

print('fed avg summary')
for attack, attack_group in fed_avg_results[fed_avg_results['attack']!='clean'].groupby('attack'):
    attack_group = attack_group[attack_group['fraction'] == '0.5']
    if attack == 'backdoor':
        print(attack, np.round(attack_group['accuracy_m'].mean()*100, 2))
        continue
    print(attack, np.round((fed_avg_clean['accuracy_m'].to_numpy() - attack_group['accuracy_m']).mean()*100, 2))

# plt.bar()
attack_data = {}
for attack, attack_group in fed_avg_results.groupby('attack'):
    y = {}
    for non_iid, non_iid_group in attack_group.groupby('non_iid'):
        y[non_iid] = non_iid_group['accuracy_m'].to_numpy() * 100
    attack_data[attack] = y

clean = attack_data['clean']
for attack in attack_data:
    if attack == 'clean' or attack == 'backdoor':
        continue
    data = attack_data[attack]
    for non_iid in clean:
        data[non_iid] = np.concatenate((clean[non_iid], data[non_iid]))

del attack_data['clean']

width = 0.22

x_ticks = ['Clean', '0.1', '0.3', '0.5']
y_label = 'Accuracy'
x_label = 'Fraction of malicious devices'
path = root + '0images/fedavg/'
os.makedirs(path, exist_ok=True)

plt.rcParams.update({'font.size': 13})

for attack in attack_data:
    fig = plt.figure()
    data = attack_data[attack]
    for i, non_iid in enumerate(data):
        accuracy_values = data[non_iid]
        x = np.arange(len(accuracy_values))
        autolabel(plt.bar(x + width * i, accuracy_values, width, label='non iid: ' + non_iid))
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,y1 ,y2 + 5))
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xticks(x + width, x_ticks[-len(x):])
    plt.legend(loc="lower left")
    file = path + attack + '.png'
    plt.savefig(file, bbox_inches='tight')
    plt.close(fig)



fed avg summary
backdoor 99.8
deletedata -0.01
labelflip 18.09
noisedata 2.34
overlapdata 0.78
randomupdate 77.45
signflip 77.45
unbalancedata -0.31


In [67]:

clean_results =  df[df['attack'] == 'clean']

# plt.bar()
clean_data = {}
for aggregator, aggregator_group in clean_results.groupby('aggregator'):
    clean_data[aggregator] = aggregator_group['accuracy_m'].to_numpy() * 100


width = 0.15


x_ticks = ['0', '0.4', '0.7']
y_label = 'Accuracy'
x_label = 'Non iid degree'

file = root + '0images/clean.png'

plt.rcParams.update({'font.size': 15})

fig = plt.figure(figsize=(10, 6))

if 'adni' in root:
    x = [aggregator for aggregator in clean_data]
    y = [clean_data[aggregator][0] for aggregator in clean_data]
    bars = plt.bar(x, y, 0.4)
    for i, bar in enumerate(bars):
        bar.set_color('C{}'.format(i))
    autolabel(bars)

else:
    for i, aggregator in enumerate(clean_data):
        accuracy_values = clean_data[aggregator]
        x = np.arange(len(accuracy_values))
        autolabel(plt.bar(x + width * i, accuracy_values, width, label=aggregator))
    plt.xticks(x + width, x_ticks[-len(x):])
    plt.xlabel(x_label)
    plt.legend(loc="lower left")

x1,x2,y1,y2 = plt.axis()
plt.axis((x1,x2,y1 ,y2 + 5))
plt.ylabel(y_label)
plt.savefig(file, bbox_inches='tight')
plt.close(fig)


In [68]:
if 'adni' in root:
    defenses_results = df[df['attack']!='clean']
else:
    defenses_results =  df[
    (df['attack']=='labelflip') |
    (df['attack'] == 'randomupdate') |
    (df['attack'] == 'signflip') |
    (df['attack'] == 'backdoor')
    ]

width = 0.2

x_ticks = ['0.1', '0.3', '0.5']
y_label = 'Accuracy'
x_label = 'Fraction of malicious devices'

path = root + '0images/defences/'
plt.rcParams.update({'font.size': 13})

for non_iid, non_iid_group in defenses_results.groupby('non_iid'):
    p = path+non_iid+'/'
    os.makedirs(p, exist_ok=True)
    for attack, attack_group in non_iid_group.groupby('attack'):
        fig = plt.figure()
        for i, (aggregator, aggregator_group) in enumerate(attack_group.groupby('aggregator')):
            accuracy_values = aggregator_group['accuracy_m'].to_numpy()*100
            x = np.arange(len(accuracy_values))
            autolabel(plt.bar(x + width * i, accuracy_values, width, label=aggregator))
        x1,x2,y1,y2 = plt.axis()
        plt.axis((x1,x2,y1 ,y2 + 5))
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xticks(x + width, x_ticks[-len(x):])
        plt.legend(loc="lower left")
        file = p + attack + '.png'
        plt.savefig(file, bbox_inches='tight')
        plt.close(fig)


In [69]:
all_confs = defenses_results
if 'adni' in root:
    all_confs = df[
    (df['attack']=='labelflip') |
    (df['attack'] == 'randomupdate') |
    (df['attack'] == 'signflip') |
    (df['attack'] == 'overlapdata') |
    (df['attack'] == 'backdoor')
    ]


print('aggregator summary')


rank_data = all_confs[all_confs['attack']!='backdoor']
ranks = defaultdict(list)
for non_iid, non_iid_group in rank_data.groupby('non_iid'):
    for attack, attack_group in non_iid_group.groupby('attack'):
        for fraction, fraction_group in attack_group.groupby('fraction'):
            fraction_group['accuracy_m']*=100
            fraction_group['accuracy_m'] = fraction_group['accuracy_m'].round()
            for aggregator, rank in zip(fraction_group['aggregator'].to_numpy(), fraction_group['accuracy_m'].rank(ascending=False, method='max').to_numpy()):
                ranks[aggregator].append(rank)
print(ranks)

for aggregator in ranks:
    print(aggregator, np.unique(ranks[aggregator], return_counts=True)[1][0])

print('non targeted attacks')
for aggregator, aggregator_group in all_confs[all_confs['attack']!='backdoor'].groupby('aggregator'):
    print(aggregator, '\t\t', np.round(aggregator_group['accuracy_m'].to_numpy().mean()*100, 2))

print('\n')

print('targeted')
for aggregator, aggregator_group in all_confs[all_confs['attack']=='backdoor'].groupby('aggregator'):
    print(aggregator, '\t\t', np.round(aggregator_group['accuracy_m'].to_numpy().mean()*100, 2))




aggregator summary
defaultdict(<class 'list'>, {'fedavg': [2.0, 3.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0], 'krum': [4.0, 4.0, 4.0, 2.0, 1.0, 1.0, 3.0, 1.0, 1.0, 4.0, 4.0, 4.0, 3.0, 2.0, 1.0, 3.0, 1.0, 1.0, 4.0, 4.0, 4.0, 3.0, 3.0, 1.0, 3.0, 1.0, 1.0], 'median': [3.0, 3.0, 3.0, 1.0, 2.0, 4.0, 1.0, 2.0, 4.0, 3.0, 3.0, 3.0, 1.0, 1.0, 4.0, 1.0, 4.0, 4.0, 3.0, 2.0, 3.0, 1.0, 1.0, 4.0, 2.0, 2.0, 4.0], 'trimmedmean': [2.0, 3.0, 2.0, 3.0, 4.0, 4.0, 2.0, 4.0, 4.0, 2.0, 2.0, 3.0, 2.0, 3.0, 4.0, 2.0, 4.0, 4.0, 2.0, 3.0, 2.0, 2.0, 2.0, 4.0, 1.0, 4.0, 4.0]})
fedavg 7
krum 10
median 7
trimmedmean 1
non targeted attacks
fedavg 		 33.51
krum 		 50.3
median 		 59.2
trimmedmean 		 50.81


targeted
fedavg 		 99.91
krum 		 2.63
median 		 82.0
trimmedmean 		 99.71


In [70]:

cifar_raw = pd.read_csv('results/cifar/0images/raw.csv')
adni_raw = pd.read_csv('results/adni/0images/raw.csv')

comb_cifar_raw = pd.read_csv('results/combination/cifar/0images/raw.csv')
comb_adni_raw = pd.read_csv('results/combination/adni/0images/raw.csv')

raw_data = {
    'cifar': {
        'best agg': cifar_raw,
        'combination': comb_cifar_raw
    },
    'adni': {
        'best agg': adni_raw,
        'combination': comb_adni_raw
    },
}


width = 0.3

x_ticks = ['0.1', '0.3', '0.5']
y_label = 'Accuracy'
x_label = 'Fraction of malicious devices'

path = 'results/'

plt.rcParams.update({'font.size': 15})

for dataset in raw_data:
    for attack in ['labelflip', 'signflip']:
        fig = plt.figure()
        for i, aggregator in enumerate(raw_data[dataset]):
            raw = raw_data[dataset][aggregator]
            raw1 = raw[(raw['attack'] == attack) & (raw['non_iid'] == 0.4)]
            if raw1.empty:
                raw1 = raw[(raw['attack'] == attack)]
            raw = raw1
            accuracy_values = []
            for fraction, fraction_group in raw.groupby('fraction'):
                accuracy_values.append(fraction_group['accuracy_m'].to_numpy().max() * 100)
            x = np.arange(len(accuracy_values))
            autolabel(plt.bar(x + width * i, accuracy_values, width, label=aggregator, color='C{}'.format(i+8)))
        x1,x2,y1,y2 = plt.axis()
        plt.axis((x1,x2,y1 ,y2 + 10))
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xticks(x + width, x_ticks[-len(x):])
        plt.legend(loc="lower left")

        p = path + dataset + '/0images/combination/'
        os.makedirs(p, exist_ok=True)

        file = p + attack + '.png'
        plt.savefig(file, bbox_inches='tight')
        plt.close(fig)